In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy
%matplotlib tk
def twos_complement(value,bits):
    if value & (1 << (bits-1)):
        value -= 1 << bits
    return value


#dat = pd.read_csv("leaf ze1 evcan3_inverter_only_hdr.txt", sep=" ",skipinitialspace=1, )
dat = pd.read_csv("drive fast1 derate.trc", sep=" ", skipinitialspace=1, header=14)

#dat.plot(x='epoch time',y=['Pack Amps','Pack Volts'])
#dat.plot(x='epoch time',y=['Pack T1 C','Motor Temp'])
#dat.plot(x='epoch time',y=['Avg CP mV'])
print(dat)
dat.iloc[:, 1]


VCM = dat[dat["PGN"] == "01D4"] 
VCM.loc[:,'0'] = VCM['0'].apply(int, base=16)
VCM.loc[:,'1'] = VCM['1'].apply(int, base=16)
VCM.loc[:,'2'] = VCM['2'].apply(int, base=16)
VCM.loc[:,'3'] = VCM['3'].apply(int, base=16)
VCM.loc[:,'4'] = VCM['4'].apply(int, base=16)
VCM.loc[:,'5'] = VCM['5'].apply(int, base=16)
VCM.loc[:,'6'] = VCM['6'].apply(int, base=16)
VCM.loc[:,'7'] = VCM['7'].apply(int, base=16)

print(VCM)

VCM['TorqueR'] = VCM['3'].apply(lambda x: x >> 4)+VCM['2']*16
VCM.loc[:,'TorqueR'] = VCM['TorqueR'].apply(twos_complement,bits=12)
VCM.loc[:,'TorqueR'] = VCM['TorqueR'] * 0.25

ax = VCM.plot(x='TIME',y=['TorqueR'])



INV = dat[dat["PGN"] == "01DA"] 
INV.loc[:,'0'] = INV['0'].apply(int, base=16)
INV.loc[:,'1'] = INV['1'].apply(int, base=16)
INV.loc[:,'2'] = INV['2'].apply(int, base=16)
INV.loc[:,'3'] = INV['3'].apply(int, base=16)
INV.loc[:,'4'] = INV['4'].apply(int, base=16)
INV.loc[:,'5'] = INV['5'].apply(int, base=16)
INV.loc[:,'6'] = INV['6'].apply(int, base=16)
INV.loc[:,'7'] = INV['7'].apply(int, base=16)

INV.loc[:,'TorqueA'] = ((INV['2']&7)*255 + INV['3'])
INV.loc[:,'TorqueA'] = INV['TorqueA'].apply(twos_complement,bits=11)
INV.loc[:,'TorqueA'] = INV['TorqueA'] * 0.5

INV.loc[:,'RPM'] = INV['5'].apply(lambda x: x >> 1)+INV['4']*127
INV.loc[:,'RPM'] = INV['RPM'].apply(twos_complement,bits=15)
INV.loc[:,'PowerAtWheels'] = INV['TorqueA'] * INV['RPM'] / 9.5488


LBC = dat[dat["PGN"] == "01DB"] 
LBC.loc[:,'0'] = LBC['0'].apply(int, base=16)
LBC.loc[:,'1'] = LBC['1'].apply(int, base=16)
LBC.loc[:,'2'] = LBC['2'].apply(int, base=16)
LBC.loc[:,'3'] = LBC['3'].apply(int, base=16)
LBC.loc[:,'4'] = LBC['4'].apply(int, base=16)
LBC.loc[:,'5'] = LBC['5'].apply(int, base=16)
LBC.loc[:,'6'] = LBC['6'].apply(int, base=16)
LBC.loc[:,'7'] = LBC['7'].apply(int, base=16)

LBC.loc[:,'Current'] = LBC['1'].apply(lambda x: x >> 5) + LBC['0']*8
LBC.loc[:,'Current'] = LBC['Current'].apply(twos_complement,bits=11)
LBC.loc[:,'Current'] = LBC['Current'] * 0.5

LBC.loc[:,'Voltage'] = LBC['1'].apply(lambda x: x >> 6) + LBC['0']*4
LBC.loc[:,'Voltage'] = LBC['Voltage'] * 0.5

LBC.loc[:,'PowerAtBatt'] = LBC['Voltage'] * LBC['Current']


ay = INV.plot(x='TIME',y=['TorqueA'],ax=ax)
INV.plot(x='TIME',y=['PowerAtWheels'])
INV.plot(x='TIME',y=['RPM'])
LBC.plot(x='TIME',y=['PowerAtBatt'])
LBC.plot(x='TIME',y=['Current'])


print(VCM)
#plt.show()

             ID        TIME  BUS  RX   PGN  -  DLC   0    1    2    3    4  \
0            1)       0.588    1  Rx  0108  -    3  00   0F   22  NaN  NaN   
1            2)       4.257    1  Rx  0284  -    8  40   2B   40   08   1E   
2            3)       6.177    1  Rx  01CB  -    7  00   00   00   00   60   
3            4)       6.433    1  Rx  011A  -    8  4E   40   04   55   00   
4            5)       6.988    1  Rx  01DA  -    8  B9   32   18   5E   2B   
...         ...         ...  ...  ..   ... ..  ...  ..  ...  ...  ...  ...   
728226  728227)  584210.120    1  Rx  01DB  -    8  FF   C0   B7   AA   00   
728227  728228)  584210.248    1  Rx  01C2  -    1  54  NaN  NaN  NaN  NaN   
728228  728229)  584210.461    1  Rx  01DC  -    8  7D   11   8F   FD   0C   
728229  728230)  584210.760    1  Rx  0108  -    3  00   0F   22  NaN  NaN   
728230  728231)  584214.386    1  Rx  01CB  -    7  00   00   48   04   60   

          5    6    7  
0       NaN  NaN  NaN  
1        FF   C

C:\Users\Jason\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


             ID        TIME  BUS  RX   PGN  -  DLC    0    1   2    3    4  \
8            9)       7.969    1  Rx  01D4  -    8  195  118  11  208   71   
18          19)      17.953    1  Rx  01D4  -    8  195  118  11  208  135   
31          32)      27.980    1  Rx  01D4  -    8  195  118  11  224  199   
42          43)      37.964    1  Rx  01D4  -    8  195  118  11  224    7   
55          56)      47.990    1  Rx  01D4  -    8  195  118  11  224   71   
...         ...         ...  ...  ..   ... ..  ...  ...  ...  ..  ...  ...   
728168  728169)  584169.758    1  Rx  01D4  -    8  251    4   0    0    7   
728182  728183)  584179.784    1  Rx  01D4  -    8  251    4   0    0   71   
728194  728195)  584189.555    1  Rx  01D4  -    8  251    4   0    0  135   
728207  728208)  584199.795    1  Rx  01D4  -    8  251    4   0    0  199   
728225  728226)  584209.864    1  Rx  01D4  -    8  251    4   0    0    7   

         5    6    7  
8       68  129  199  
18      68  129  

<ipython-input-1-fac0b27893cf>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VCM['TorqueR'] = VCM['3'].apply(lambda x: x >> 4)+VCM['2']*16
C:\Users\Jason\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\Jason\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


             ID        TIME  BUS  RX   PGN  -  DLC    0    1   2    3    4  \
8            9)       7.969    1  Rx  01D4  -    8  195  118  11  208   71   
18          19)      17.953    1  Rx  01D4  -    8  195  118  11  208  135   
31          32)      27.980    1  Rx  01D4  -    8  195  118  11  224  199   
42          43)      37.964    1  Rx  01D4  -    8  195  118  11  224    7   
55          56)      47.990    1  Rx  01D4  -    8  195  118  11  224   71   
...         ...         ...  ...  ..   ... ..  ...  ...  ...  ..  ...  ...   
728168  728169)  584169.758    1  Rx  01D4  -    8  251    4   0    0    7   
728182  728183)  584179.784    1  Rx  01D4  -    8  251    4   0    0   71   
728194  728195)  584189.555    1  Rx  01D4  -    8  251    4   0    0  135   
728207  728208)  584199.795    1  Rx  01D4  -    8  251    4   0    0  199   
728225  728226)  584209.864    1  Rx  01D4  -    8  251    4   0    0    7   

         5    6    7  TorqueR  
8       68  129  199    47.25  